# Data Acquisition

THis notebook demonstrates the process followed when taking a scan

In [1]:
# Importing the required libraries
import glob, serial, time, math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from skimage.transform import iradon
from IPython.display import display

The microcontroller acts as a serial port, and we will send commands over a serial connection. Here, we list the ports found by the computer so that we know where to find our teensy:

In [2]:
ports = glob.glob('/dev/tty[A-Za-z]*') # Modify if on a platform other than Linux
ports[:5]

['/dev/ttyUSB2',
 '/dev/ttyUSB1',
 '/dev/ttyUSB0',
 '/dev/ttyprintk',
 '/dev/ttyS31']

In [3]:
# Open a serial conncetion
ser = serial.Serial(ports[0], 115200) # Usually the first. /dev/ttyACM0 or similar

SerialException: [Errno 16] could not open port /dev/ttyUSB2: [Errno 16] Device or resource busy: '/dev/ttyUSB2'

In [4]:
# Some utility functions
a_steps = 640 # Steps per revolution - 200 * 80/25
b_steps = 1280 # 160 teeth -> 200*160/25

apos = 0
bpos = 0

read = ''

def step_a(n):
    message = b'a'
    if n < 0:
        n = -n
        message = b'A'
    for i in range(n):
        ser.write(message)
        while ser.in_waiting < 1:
            pass # wait for a response
        read = ser.read_all()
#         print(read)
    return n

def set_a(angle):
    step_coords = int(angle/(2*math.pi) * a_steps) # Angle in radians
    step_a(step_coords - apos)
    return step_coords

def step_b(n):
    message = b'b'
    if n < 0:
        n = -n
        message = b'B'
    for i in range(n):
        ser.write(message)
        while ser.in_waiting < 1:
            pass # wait for a response
        read = ser.read_all()
#         print(read)
    return n

def set_b(angle):
    step_coords = int(angle/(2*math.pi) * b_steps) # Angle in radians
    step_b(step_coords - bpos)
    return step_coords

def read_adc():
    ser.write(b'E')
    while ser.in_waiting < 1:
        pass # wait for a response
    return ser.read_all()

The first thing we do is take a set of readings with nothing on the scanner - this is a baseline and lets us account for LED spread, ambient light and so on. 

In [5]:
fn = 'test' # The file name to save data under

In [6]:
# Taking one set of readings as a baseline (no object)
step_b(-200)
base = []
for b in range(400):
    base.append(int(read_adc().strip()))
    step_b(1)
step_b(-200)
                
plt.plot(base)
b = pd.DataFrame(base)
b.to_csv(fn + 'base.csv', index = False)

NameError: name 'ser' is not defined

Now we place an object on the scanner and repeat, rotating the object, taking a set of readings, rotating and so on

In [7]:
# Now, doing these scans from several angles. Doing 32 As (views/thetas)
from IPython.display import clear_output
readings = []
nv=32
nd = 400 # Hard coded for now
for a in range(nv):
    line = []
    step_b(-200)
    for b in range(400):
        line.append(int(read_adc().strip()))
        step_b(1)
    readings.append(line)
    step_a(int(-640/nv)) # A has 640 steps in 360 degrees.
    step_b(-200) # return to start

# Save the scan
d = pd.DataFrame(readings)
d.to_csv(fn+'_' + str(nv) + 'v_' + str(nd) + 'd.csv', index=False)

NameError: name 'ser' is not defined

At this stage, our data has been gathered and we can reconstruct the image or analyse it in some other way. See Summary - Image Reconstruction for details on this